In [ ]:
from y0.dsl import A, B, C, D, One, P, Product, Q, R, S, Sum, T, Variable, W, X, Y, Z, Zero

# Level 1 of Pearl's causal hierarchy (conditioning, marginalization)

Probability $1$ event

In [41]:
One()

One()

Probability $0$ event

In [42]:
Zero()

Zero()

 Probability of event $A$ occurring

In [43]:
P(A)

P(A)

A variable can be created with the `y0.dsl.Variable` class. Note that it is clever about subscripting the name if it ends with numbers.

In [44]:
A1 = Variable("A1")
P(A1)

P(A1)

Probability of event $A$ not occuring 

In [45]:
P(~A)

P(+A)

Multiplication property of zero

In [46]:
Zero() * P(A, B)

Zero()

In [47]:
P(A, B) * Zero()

Zero()

Multiplicative identity

In [48]:
P(A, B) * One()

P(A, B)

In [49]:
One() * P(A, B)

P(A, B)

Joint probability of $A$ and $B$

In [50]:
P(A, B)

P(A, B)

Probability of event $A$ given event $B$

In [51]:
P(A | B)

P(A | B)

Probability of event $A$ not occuring given $B$

In [52]:
P(~A | B)

P(+A | B)

Probability of event $A$ given $B$ does not occur

In [53]:
P(A | ~B)

P(A | +B)

Probability event $A$ does not occur given $B$ does not occur

In [54]:
P(~A | ~B)

P(+A | +B)

Marginal Probability that event $B$ occurs in a joint distribution over $A$ and $B$.

In [55]:
Sum[A](P(A, B))

Sum[A](P(A, B))

Conditional probability of a joint distribution of $A$ and $B$ given $B$

In [56]:
P(A, B) / Sum[B](P(A, B))

((P(A, B) / Sum[B](P(A, B))))

Inverse probability of $A$ given $B$

In [57]:
One() / P(A | B)

((One() / P(A | B)))

Ratio of probabilities

In [58]:
(One() / P(A | B)) * P(A)

((P(A) / P(A | B)))

Product of probabilities

In [59]:
P(A | B) * P(B)

P(A | B) * P(B)

Complex probabilistic expressions 

In [60]:
Sum[B](P(A | B))

Sum[B](P(A | B))

In [61]:
Sum[S, T](P(A | B) * Sum[R](P(C | D)))

Sum[S, T](P(A | B) * Sum[R](P(C | D)))

In [62]:
Sum[W](P((Y @ ~Z @ W) & X) * P(D) * P(Z @ D) * P(W @ ~X))

Sum[W](P(D) * P[+X](W) * P(X, Y @ (-W, +Z)) * P[D](Z))

In [63]:
(P(A) / P(B)) / (P(C) / P(D))

((P(A) * P(D) / P(B) * P(C)))

Probabilistic parameter

In [ ]:
Q[A](B)

In [ ]:
P(Variable(v) for v in ["A", "B", "C"])

In [ ]:
Product([P(A), P(B), P(C)])

In [ ]:
Sum(P(Variable(v) for v in ["A", "B", "C"]), frozenset(Variable(v) for v in ["A", "B"]))

# Level 2 of Pearl's causal hierarchy (intervention)

Probability of $Y$ given $do(X)$

In [64]:
P[X](Y, Z)

P[X](Y, Z)

In [65]:
P(Y @ X, Z @ X)

P[X](Y, Z)

In [ ]:
P(Y @ X)

Post-interventional conditional distribution of $Y$ given $Z$

In [ ]:
P[X](Y | Z)

# Level 3 of Pearl's causal hierarchy (counterfactual)

Probability of sufficient causation

In [66]:
P(+Y @ +X | -X, -Y)

P(+Y @ +X | -X, -Y)

Probability of necessary causation

In [67]:
P(-Y @ -X | +X, +Y)

P(-Y @ -X | +X, +Y)

Probability of necessary and sufficent causation

In [68]:
P(+Y @ +X, -Y @ -X)

P(+Y @ +X, -Y @ -X)

Multiple interventions

In [69]:
P(A @ ~W @ X)

P[+W,X](A)

In [70]:
P(A @ (~W, X))

P[+W,X](A)

In [77]:
P(A, X @ ~W)

P(A, X @ +W)